In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import time
import os

import pandas as pd

In [25]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\napha\.wdm\drivers\chromedriver\win32\99.0.4844.51]


In [26]:
data = pd.read_csv('./Rayong/test/rayong_test.csv')
data.columns = ["date","PM2.5"]
data.head()

,date,PM2.5
0,2020-07-01 00:00:00,5.0
1,2020-07-01 01:00:00,3.0
2,2020-07-01 02:00:00,2.0
3,2020-07-01 03:00:00,4.0
4,2020-07-01 04:00:00,3.0


In [27]:
from datetime import datetime, timedelta
data['date_utc'] = pd.to_datetime(data['date']) - timedelta(hours=7)
data['url_date'] = data['date_utc'].apply(lambda x : x.strftime("%Y/%m/%d/%H%M")+"Z")
data.head()

,date,PM2.5,date_utc,url_date
0,2020-07-01 00:00:00,5.0,2020-06-30 17:00:00,2020/06/30/1700Z
1,2020-07-01 01:00:00,3.0,2020-06-30 18:00:00,2020/06/30/1800Z
2,2020-07-01 02:00:00,2.0,2020-06-30 19:00:00,2020/06/30/1900Z
3,2020-07-01 03:00:00,4.0,2020-06-30 20:00:00,2020/06/30/2000Z
4,2020-07-01 04:00:00,3.0,2020-06-30 21:00:00,2020/06/30/2100Z


In [28]:
from selenium.webdriver.common.by import By

In [29]:
def getTemp(lat,long,date):
    url = 'https://earth.nullschool.net/#'+date+'/wind/surface/level/anim=off/overlay=temp/orthographic/loc='+long+','+lat
    driver.get(url=url)
    time.sleep(2)
    html = driver.execute_script("return document.documentElement.outerHTML")

    all_link = driver.find_elements(by=By.CSS_SELECTOR, value='#spotlight-panel.panel.card')
    for link in all_link:
        # print( link.text.split("\n")[-2])
        return link.text.split("\n")[-2]

In [31]:
def getWind(lat,long,date):
    url = 'https://earth.nullschool.net/#'+date+'/wind/isobaric/850hPa/anim=off/orthographic/loc='+long+','+lat
    driver.get(url=url)
    time.sleep(2)
    html = driver.execute_script("return document.documentElement.outerHTML")

    all_link = driver.find_elements(by=By.CSS_SELECTOR, value='#spotlight-panel.panel.card')
    for link in all_link:
        # print(link.text.split("\n")[2])
        return link.text.split("\n")[2]

In [38]:
data['temp'] = data['url_date'].apply(lambda x : getTemp('12.671521','101.275875',x))

In [ ]:
data['wind'] = data['url_date'].apply(lambda x : getWind('12.671521','101.275875',x))

In [ ]:
data.to_csv('scrapeRayong.csv')